In [1]:
print('Hello world!')

Hello world!


In [2]:
print('This is a git commit test.')

This is a git commit test.


This section is to document the cleaning steps for the data files in the dataset folder. This duplicates my cleaning steps I performed in Excel for the Coursera Case Study 1. The final report for that study can be found on my [Kaggle site](https://www.kaggle.com/code/scwilso28/coursera-case-study-1-cyclistic-data-analysis).

In [3]:
import pandas as pd
import zipfile

In [4]:
zf = zipfile.ZipFile('Datasets/202103-divvy-tripdata.zip')
df = pd.read_csv(zf.open('202103-divvy-tripdata.csv'))

In [5]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228496 entries, 0 to 228495
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             228496 non-null  object 
 1   rideable_type       228496 non-null  object 
 2   started_at          228496 non-null  object 
 3   ended_at            228496 non-null  object 
 4   start_station_name  213648 non-null  object 
 5   start_station_id    213648 non-null  object 
 6   end_station_name    211769 non-null  object 
 7   end_station_id      211769 non-null  object 
 8   start_lat           228496 non-null  float64
 9   start_lng           228496 non-null  float64
 10  end_lat             228329 non-null  float64
 11  end_lng             228329 non-null  float64
 12  member_casual       228496 non-null  object 
dtypes: float64(4), object(9)
memory usage: 146.0 MB


In [6]:
to_drop = ['start_station_id', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng']
df.drop(to_drop, inplace = True, axis = 1)

In [7]:
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

In [8]:
df['member_casual'] = df['member_casual'].map({'casual':0, 'member':1})
df.info(memory_usage="deep")
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228496 entries, 0 to 228495
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   ride_id             228496 non-null  object        
 1   rideable_type       228496 non-null  object        
 2   started_at          228496 non-null  datetime64[ns]
 3   ended_at            228496 non-null  datetime64[ns]
 4   start_station_name  213648 non-null  object        
 5   end_station_name    211769 non-null  object        
 6   member_casual       228496 non-null  int64         
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 69.9 MB


,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual
0,CFA86D4455AA1030,classic_bike,2021-03-16 08:32:30,2021-03-16 08:36:34,Humboldt Blvd & Armitage Ave,Stave St & Armitage Ave,0
1,30D9DC61227D1AF3,classic_bike,2021-03-28 01:26:28,2021-03-28 01:36:55,Humboldt Blvd & Armitage Ave,Central Park Ave & Bloomingdale Ave,0
2,846D87A15682A284,classic_bike,2021-03-11 21:17:29,2021-03-11 21:33:53,Shields Ave & 28th Pl,Halsted St & 35th St,0
3,994D05AA75A168F2,classic_bike,2021-03-11 13:26:42,2021-03-11 13:55:41,Winthrop Ave & Lawrence Ave,Broadway & Sheridan Rd,0
4,DF7464FBE92D8308,classic_bike,2021-03-21 09:09:37,2021-03-21 09:27:33,Glenwood Ave & Touhy Ave,Chicago Ave & Sheridan Rd,0


In [9]:
duplicate_check = ['rideable_type', 'started_at', 'ended_at', 'start_station_name', 'end_station_name', 'member_casual']
duplicate_entries = df[df.duplicated(subset = duplicate_check, keep = False)]
len(duplicate_entries)/2

26.0

In [10]:
100*(len(duplicate_entries)/2)/len(df)

0.011378754989146419

In [11]:
len(df)

228496

In [12]:
df.drop_duplicates(subset = duplicate_check, inplace=True)
len(df)

228470